In [13]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                           f1_score, roc_auc_score, mean_squared_error, r2_score)
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import os

# ==================== 参数配置 ====================
input_path = "归一化数据-20251120.xlsx"  # 输入数据路径
output_dir = "xgboost_model_ncv"     # 输出目录
target_column = "SO2 tolerance"     # 目标变量列名
outer_test_size = 0.2               # 外层测试集比例
inner_test_size=0.2                 # 内层测试集比例   
random_state = 42                   # 随机种子
n_splits = 5                        # 交叉验证折数
model_path = os.path.join(output_dir, "xgboost_model.pkl")  # 模型保存路径

# ==================== 函数定义 ====================
def prepare_data(input_path, target_column):
    """数据加载与预处理"""
    df = pd.read_excel(input_path)
    if target_column not in df.columns:
        raise ValueError(f"目标列 {target_column} 不存在于数据中")
    
    y = df[target_column]
    X = df.drop(columns=[target_column])
    numeric_cols = X.select_dtypes(include=['number']).columns.tolist()
    return X[numeric_cols], y

def determine_problem_type(y):
    """自动判断问题类型"""
    if y.nunique() <= 10:  # 分类问题（类别数≤10）
        return ('binary:logistic' if y.nunique() == 2 else 'multi:softprob', 
                "classification")
    return ('reg:squarederror', "regression")

def get_model_params(objective, random_state):
    """获取模型参数"""
    return {
        'objective': objective,
        'eval_metric': 'logloss' if objective.startswith('binary') else 'rmse',
        'eta': 0.1,
        'max_depth': 6,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'seed': random_state,
        'n_jobs': -1
    }

def train_and_evaluate(X_train, y_train, X_test, y_test, params, problem_type):
    """训练模型并返回评估结果"""
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)
    
    model = xgb.train(
        params,
        dtrain,
        num_boost_round=1000,
        evals=[(dtrain, 'train'), (dtest, 'test')],
        early_stopping_rounds=50,
        verbose_eval=False
    )
    
    # 评估模型
    ddata_train = xgb.DMatrix(X_train)
    ddata_test = xgb.DMatrix(X_test)
    
    if problem_type == "classification":
        y_pred_train =np.round(model.predict(ddata_train)) if params['objective'] == 'binary:logistic' else np.argmax(model.predict(ddata_train), axis=1)
        y_pred_test =np.round(model.predict(ddata_test)) if params['objective'] == 'binary:logistic' else np.argmax(model.predict(ddata_test), axis=1)
        metrics = {
            'Train Accuracy': accuracy_score(y_train, y_pred_train),
            'Test Accuracy': accuracy_score(y_test, y_pred_test),
            'Train F1': f1_score(y_train, y_pred_train, average='macro'),
            'Test F1': f1_score(y_test, y_pred_test, average='macro')
        }
    else:
        y_pred_train = model.predict(ddata_train)
        y_pred_test = model.predict(ddata_test)
        metrics = {
            'Train R2': r2_score(y_train, y_pred_train),
            'Test R2': r2_score(y_test, y_pred_test),
            'Train RMSE': np.sqrt(mean_squared_error(y_train, y_pred_train)),
            'Test RMSE': np.sqrt(mean_squared_error(y_test, y_pred_test))
        }
    return model, metrics
def plot_feature_importance(model, output_dir):
    """
    绘制特征重要性图（使用自定义颜色方案）
    
    参数:
        model: 训练好的XGBoost模型
        output_dir: 输出目录
    """
    # 自定义颜色方案（根据用户提供的颜色）
    custom_colors = {
        'EN.': '#82bbf0', 
        'P.Tim. ':'#dff1ff', 
        'S.A.': '#FDB3CC', 
        'GHSV': '#ff78A8', 
        'I.R.': '#82bbf0', 
        'I.E.': '#82bbf0', 
        'P.D.': '#FDB3CC',
        'H2O': '#ff78A8', 
        'P.TEM.': '#dff1ff', 
        'M.P.': '#82bbf0', 
        'T.C.': '#82bbf0', 
        'P.V.': '#FDB3CC', 
        'C.Tem.': '#FDB3CC',
        'Density': '#82bbf0',
        'NH3': '#ff78A8', 
        'SO2': '#dff1ff', 
        'C.Tim.': '#FDB3CC', 
        'O2': '#ff78A8', 
        'NO': '#ff78A8'
    }
    
    # 设置全局字体样式
    plt.rcParams['font.family'] = 'Arial'
    plt.rcParams['font.weight'] = 'bold'
    
    # 设置图形大小和边缘距离
    fig, ax = plt.subplots(figsize=(14, 10), facecolor='none')
    plt.subplots_adjust(left=0.2, right=0.95, top=0.9, bottom=0.1)
    ax.grid(False)
    ax.set_facecolor('white')  
    
    # 获取特征重要性数据
    importance = model.get_score(importance_type='weight')
    importance = sorted(importance.items(), key=lambda x: x[1], reverse=False)
    feature_names = [x[0] for x in importance]
    feature_imp = [x[1] for x in importance]
    
    # 为每个特征分配颜色（如果特征未在自定义颜色中定义，使用默认颜色）
    colors = [custom_colors.get(feature, '#1f77b4') for feature in feature_names]
    
    # 绘制水平条形图
    bars = ax.barh(
        range(len(feature_names)), 
        feature_imp, 
        height=0.8,  # 调整柱形宽度
        color=colors,  # 使用自定义颜色
    )
    
    # 设置y轴刻度
    ax.set_yticks(range(len(feature_names)))
    ax.set_yticklabels(feature_names)
    
    # 设置x轴标签
    plt.xlabel(
        "Importance",
        fontdict={
            'fontname': 'Arial',
            'fontsize': 24,
            'fontweight': 'bold'
        },
        labelpad=10  # 标签与轴的距离
    )
    
    # 设置y轴标签
    plt.ylabel(
        ' ',
        fontdict={
            'fontname': 'Arial',
            'fontsize': 20,
            'fontweight': 'bold'
        },
        labelpad=10  # 标签与轴的距离
    )
    
    # 设置坐标轴刻度字体
    for label in ax.get_xticklabels() + ax.get_yticklabels():
        label.set_fontname('Arial')
        label.set_fontsize(20)
    
    # 保留所有边框并设置样式
    for spine in ax.spines.values():
        spine.set_visible(True)
        spine.set_linewidth(2.5)  # 加粗边框线
        spine.set_color('black')  # 设置边框颜色
    
    # 添加数值标签
    for bar in bars:
        width = bar.get_width()
        ax.text(
            width + max(feature_imp)*0.01,  # 位置微调
            bar.get_y() + bar.get_height()/2,
            f'{width:.1f}',
            ha='left',
            va='center',
            fontname='Arial',
            fontsize=18,
            color='black'  # 确保文字颜色清晰可见
        )
    
    # 保存图像
    plt.savefig(
        os.path.join(output_dir, "feature_importance.png"), 
        dpi=300, 
        bbox_inches='tight',
        transparent=False
    )
    plt.close()
    
def plot_actual_vs_predicted(X_train, y_train, X_test, y_test, model, output_dir):
    """绘制实际值 vs 预测值图（回归问题）"""
    # 设置全局字体样式
    plt.rcParams['font.family'] = 'Arial'
    plt.rcParams['font.weight'] = 'bold'
    
    d_train = xgb.DMatrix(X_train)
    pp_tr = model.predict(d_train)
    d_test = xgb.DMatrix(X_test)
    y_predicted = model.predict(d_test)
    
    plt.figure(figsize=(10, 8))
    g = sns.JointGrid(x=y_test, y=y_predicted, height=8, space=0)

   
    g.set_axis_labels("Actual value", "Predicted value", fontsize =10, fontname = 'Arial')
    sns.scatterplot(x=y_train, y=pp_tr, s=100, color='#DB7987', ax=g.ax_joint, label='Training')
    sns.scatterplot(x=y_test, y=y_predicted, s=100, color='#82BBF0', ax=g.ax_joint, label='Testing')
    sns.regplot(x=y_test, y=y_predicted, ax=g.ax_joint, scatter=False, color='gray')
       
    # 调整坐标轴刻度标签样式
    for ax in [g.ax_joint, g.ax_marg_x, g.ax_marg_y]:
        ax.tick_params(
            axis='both', 
            which='major', 
            labelsize=28,      # 刻度标签大小
            colors='black',   # 刻度标签颜色
            width=3,
            length=8
        )
    # 设置图例样式
    g.ax_joint.legend(
        title_fontsize=28,      # 标题字体大小
        fontsize=28,           # 项目字体大小
        frameon=False,          # 显示边框
        framealpha=1,        # 边框透明度
        edgecolor='white',   # 边框颜色
        facecolor='white'      # 背景颜色
        
    )
     # 主图边框（闭合，线宽3）
    for spine in g.ax_joint.spines.values():
        spine.set_visible(True)
        spine.set_edgecolor('#333333')
        spine.set_linewidth(2.5)
    
    # 计算实际值和预测值的全局范围
    all_values = np.concatenate([y_train, y_test, pp_tr, y_predicted])
    min_val = np.min(all_values)
    max_val = np.max(all_values)
    buffer = (max_val - min_val) * 0.15  # 增加缓冲区，确保边缘数据有足够空间
    
    # 设置主图坐标范围
    g.ax_joint.set_xlim(min_val - buffer, max_val + buffer)
    g.ax_joint.set_ylim(min_val - buffer, max_val + buffer)
    
    # 绘制理想预测线
    g.ax_joint.plot([min_val, max_val], [min_val, max_val], '--', color='gray', alpha=0.5)
    
    # 优化边缘分布的面积图
    # X轴面积图 - 使用更合适的带宽和范围
    sns.kdeplot(
        x=y_train, ax=g.ax_marg_x, color='#DB7987', alpha=0.5,
        fill=True, linewidth=0, label='Training', bw_adjust=1.5,
        clip=(min_val - buffer, max_val + buffer)
    )
    sns.kdeplot(
        x=y_test, ax=g.ax_marg_x, color='#82BBF0', alpha=0.5,
        fill=True, linewidth=0, label='Testing', bw_adjust=1.5,
        clip=(min_val - buffer, max_val + buffer)
    )
    
    # y轴面积图 - 使用更合适的带宽和范围
    sns.kdeplot(
        y=pp_tr, ax=g.ax_marg_y, color='#DB7987', alpha=0.5,
        fill=True, linewidth=0, label='Training', bw_adjust=1.5,
        clip=(min_val - buffer, max_val + buffer)
    )
    sns.kdeplot(
        y=y_predicted, ax=g.ax_marg_y, color='#82BBF0', alpha=0.5,
        fill=True, linewidth=0, label='Testing', bw_adjust=1.5,
        clip=(min_val - buffer, max_val + buffer)
    )
    
    # 隐藏边缘分布的图例
    legend_x = g.ax_marg_x.legend()
    legend_x.set_visible(False)
    legend_y = g.ax_marg_y.legend()
    legend_y.set_visible(False)
    
    # 调整边缘分布坐标轴粗细
    for ax in [g.ax_marg_x, g.ax_marg_y]:
        for spine in ax.spines.values():
            spine.set_linewidth(1)
    
    # 确保边缘分布与主图坐标范围一致
    g.ax_marg_x.set_xlim(g.ax_joint.get_xlim())
    g.ax_marg_y.set_ylim(g.ax_joint.get_ylim())
    
    # 关键修改：在所有绘图操作完成后设置轴标签
    g.ax_joint.set_xlabel("Actual value", fontsize=32, fontname='Arial', fontweight='bold')
    g.ax_joint.set_ylabel("Predicted value", fontsize=32, fontname='Arial', fontweight='bold')
    
    # 调整整体布局，确保有足够空间显示面积图
    plt.subplots_adjust(top=0.92, bottom=0.08, left=0.12, right=0.95)
    
    # 保存图像
    plot_path = os.path.join(output_dir, "actual_vs_predicted.png")
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    plt.close()
    return plot_path

# ==================== 主程序 ====================
def main():
    # 创建输出目录
    os.makedirs(output_dir, exist_ok=True)
    
    # 1. 数据准备
    X, y = prepare_data(input_path, target_column)
    objective, problem_type = determine_problem_type(y)
    params = get_model_params(objective, random_state)
    
    # 2. 外层数据划分
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=outer_test_size, random_state=random_state)
    
    # 3. 外层模型训练与评估
    print("外层模型训练与评估...")
    outer_model, outer_metrics = train_and_evaluate(
        X_train, y_train, X_test, y_test, params, problem_type)
    joblib.dump(outer_model, model_path)
    print(f"\n外层模型已保存至：{model_path}")
    
    # 打印外层评估结果
    print("\n外层模型性能：")
    print(pd.DataFrame([outer_metrics], index=['Value']))
    
    # 4. 内层5折交叉验证
    print(f"\n开始内层{n_splits}折交叉验证...")
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    cv_metrics = []
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(X_train), 1):
        print(f"\n正在处理第 {fold} 折...")
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
        
        model, metrics = train_and_evaluate(X_tr, y_tr, X_val, y_val, params, problem_type)
        cv_metrics.append(metrics)
        print(f"第 {fold} 折完成 - 验证集R2: {metrics.get('Test R2', metrics.get('Test Accuracy')):.4f}")
    
    # 5. 交叉验证结果分析
    cv_results = pd.DataFrame(cv_metrics)
    print("\n交叉验证结果汇总：")
    print(cv_results)
    
    mean_metrics = cv_results.mean().to_dict()
    std_metrics = cv_results.std().to_dict()
    
    print("\n交叉验证平均性能：")
    for metric in mean_metrics:
        print(f"{metric}: {mean_metrics[metric]:.4f} ± {std_metrics[metric]:.4f}")
    
    # 6. 可视化与结果保存
    plot_feature_importance(outer_model, output_dir)
    
    if problem_type == "regression":
        plot_path = plot_actual_vs_predicted(X_train, y_train, X_test, y_test, 
                                           outer_model, output_dir)
        print(f"\n实际值 vs 预测值图已保存至：{plot_path}")
    
    # 7. 保存评估结果
    results = {
        'outer_test_r2': outer_metrics.get('Test R2', outer_metrics.get('Test Accuracy')),
        'cv_mean_r2': mean_metrics.get('Test R2', mean_metrics.get('Test Accuracy')),
        'cv_std_r2': std_metrics.get('Test R2', std_metrics.get('Test Accuracy'))
    }
    
    with open(os.path.join(output_dir, "results_summary.txt"), 'w') as f:
        f.write("模型评估结果汇总:\n")
        f.write(f"外层测试集R2: {results['outer_test_r2']:.4f}\n")
        f.write(f"交叉验证平均R2: {results['cv_mean_r2']:.4f} ± {results['cv_std_r2']:.4f}\n")
    
    print("\n评估结果已保存至 results_summary.txt")
    print("\n嵌套交叉验证完成！所有结果保存在目录：", output_dir)

if __name__ == "__main__":
    main()   


外层模型训练与评估...

外层模型已保存至：xgboost_model_ncv\xgboost_model.pkl

外层模型性能：
       Train R2  Test R2  Train RMSE  Test RMSE
Value  0.998865  0.77791    0.008845   0.152524

开始内层5折交叉验证...

正在处理第 1 折...
第 1 折完成 - 验证集R2: 0.0832

正在处理第 2 折...
第 2 折完成 - 验证集R2: 0.0362

正在处理第 3 折...
第 3 折完成 - 验证集R2: 0.4582

正在处理第 4 折...
第 4 折完成 - 验证集R2: -0.0051

正在处理第 5 折...
第 5 折完成 - 验证集R2: 0.2718

交叉验证结果汇总：
   Train R2   Test R2  Train RMSE  Test RMSE
0  0.972181  0.083182    0.047087   0.155161
1  0.994045  0.036184    0.015544   0.400615
2  0.999504  0.458175    0.006030   0.160132
3  0.980205 -0.005097    0.038352   0.216736
4  0.997062  0.271756    0.014897   0.171832

交叉验证平均性能：
Train R2: 0.9886 ± 0.0118
Test R2: 0.1688 ± 0.1933
Train RMSE: 0.0244 ± 0.0174
Test RMSE: 0.2209 ± 0.1034

实际值 vs 预测值图已保存至：xgboost_model_ncv\actual_vs_predicted.png

评估结果已保存至 results_summary.txt

嵌套交叉验证完成！所有结果保存在目录： xgboost_model_ncv


<Figure size 1000x800 with 0 Axes>

In [27]:
# -*- coding: utf-8 -*-
import shap
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import os
from matplotlib import rcParams

# ==================== 参数配置 ====================
model_path = "xgboost_model_ncv/xgboost_model.pkl"  # 模型路径
input_path = "归一化数据-20251120.xlsx"                      # 输入数据路径
output_dir = "shap_analysis"                       # 输出目录
target_column = "SO2 tolerance"                   # 目标变量列名
random_state = 42                                 # 随机种子
sample_size = 100                                 # 指定分析的样本数量


# ==================== 函数定义 ====================
def load_data_and_model():
    """加载数据和模型"""
    model = joblib.load(model_path)
    df = pd.read_excel(input_path)
    X = df.drop(columns=[target_column])
    y = df[target_column]
    numeric_cols = X.select_dtypes(include=['number']).columns.tolist()
    return X[numeric_cols], y, model

def prepare_shap_data(X, y):
    """准备SHAP分析数据，只选择前N条记录"""
    # 使用 .iloc[:n] 来获取前n条数据
    return X.iloc[:sample_size]

def plot_shap_summary(shap_values, X, output_dir):
    """绘制SHAP摘要图"""
    plt.figure(figsize=(14, 10), facecolor='white')
    
    # 计算合理的x轴范围
    max_shap = max(abs(shap_values.min()), abs(shap_values.max()))
    xlim = (-max_shap * 1.1, max_shap * 1.1)  # 使用数据本身的范围，而不是固定值
    
    shap.summary_plot(shap_values, X, show=False, plot_size=None)
    
    ax = plt.gca()
    ax.grid(False)
    ax.set_facecolor('white')
    ax.set_xlim(xlim)  # 设置对称的x轴范围
    
    # 设置标题样式
    plt.title(" ",
              fontsize=26,
              pad=20,
              fontname='Arial',
              fontweight='bold')

    # ===== 坐标轴标签设置 =====
    ax.set_xlabel("SHAP value",
                  fontsize=26,
                  fontname='Arial',
                  fontweight='bold',
                  labelpad=10)

    ax.set_ylabel(" ",
                  fontsize=22,
                  fontname='Arial',
                  fontweight='bold',
                  labelpad=10)

    # ===== 刻度标签设置 =====
    ax.tick_params(axis='x',
                   which='both',
                   labelsize=22,
                   width=3,
                   length=8,
                   direction='out')

    ax.tick_params(axis='y',
                   which='both',
                   labelsize=22,
                   width=3,
                   length=8,
                   direction='out')

    # 设置坐标轴样式
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(True)
    ax.spines['bottom'].set_color('black')
    ax.spines['bottom'].set_linewidth(2)
    
    # 保存图形
    plot_path = os.path.join(output_dir, "shap_summary.png")
    plt.savefig(plot_path, dpi=300, bbox_inches='tight', transparent=True)
    plt.close()
    return plot_path

def plot_shap_bar(shap_values, X, output_dir):
    """绘制SHAP条形图"""
    plt.figure(figsize=(12, 8), facecolor='white')
    
    shap.summary_plot(shap_values, X, plot_type="bar", show=False)
    
    ax = plt.gca()
    ax.grid(False)
    ax.set_facecolor('white')
    
    # 调整边距
    plt.subplots_adjust(left=0.3, right=0.95)
    
    # 设置坐标轴样式
    for spine in ax.spines.values():
        spine.set_visible(True)
        spine.set_linewidth(3)
    
    plot_path = os.path.join(output_dir, "shap_bar.png")
    plt.savefig(plot_path, dpi=300, bbox_inches='tight', transparent=True)
    plt.close()
    return plot_path

def plot_shap_dependence(shap_values, X, output_dir):
    """绘制SHAP依赖图"""
    os.makedirs(os.path.join(output_dir, "dependence_plots"), exist_ok=True)
    plot_paths = []
    
    for feature in X.columns:
        plt.figure(figsize=(10, 6), facecolor='white')
        shap.dependence_plot(feature, shap_values, X, show=False)
        
        ax = plt.gca()
        ax.grid(False)
        ax.set_facecolor('white')
        # 设置坐标轴样式
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['left'].set_visible(False)
        ax.spines['bottom'].set_visible(True)
        ax.spines['bottom'].set_color('black')
        ax.spines['bottom'].set_linewidth(3)
        
        plot_path = os.path.join(output_dir, "dependence_plots", f"shap_dependence_{feature}.png")
        plt.savefig(plot_path, dpi=300, bbox_inches='tight', transparent=True)
        plt.close()
        plot_paths.append(plot_path)
        
    
    return plot_paths

def plot_shap_waterfall(explainer, shap_values, X_sample, sample_index, output_dir):
    """绘制SHAP瀑布图"""
    os.makedirs(os.path.join(output_dir, "waterfall_plots"), exist_ok=True)

    plt.figure(figsize=(12, 10), facecolor='white')
    
    expected_value = explainer.expected_value
    if isinstance(expected_value, list):
        expected_value = expected_value[0]
    
    if isinstance(shap_values, list):
        shap_vals = shap_values[0]
    else:
        shap_vals = shap_values
    
    # 检查样本索引是否有效
    if sample_index >= len(X_sample):
        print(f"Warning: Sample index {sample_index} out of range, using last sample")
        sample_index = len(X_sample) - 1
    
    # 生成瀑布图
    shap.plots.waterfall(
        shap.Explanation(
            values=shap_vals[sample_index],
            base_values=expected_value,
            data=X_sample.iloc[sample_index],
            feature_names=X_sample.columns.tolist()
        ),
        show=False,
        max_display=10  # 限制显示的特征数量
    )

    ax = plt.gca()
    ax.grid(False)
    ax.set_facecolor('white')

    # 设置字体样式
    font_props = {
        'family': 'Arial',  # 字体类型
        'weight': 'bold',   # 字体加粗
        'size': 14          # 字体大小
    }
    
    # 设置坐标轴标签样式
    ax.set_xlabel(ax.get_xlabel(), fontdict=font_props)
    ax.set_ylabel(ax.get_ylabel(), fontdict=font_props)
    
    # 设置刻度标签样式
    ax.tick_params(axis='both', which='major', labelsize=14)
    for label in ax.get_xticklabels():
        label.set_fontweight('bold')
        label.set_fontname('Arial')
    for label in ax.get_yticklabels():
        label.set_fontweight('bold')
        label.set_fontname('Arial')
    
    # 设置标题样式（如果有）
    if ax.get_title():
        ax.set_title(ax.get_title(), fontdict={
            'fontsize': 14,
            'fontweight': 'bold',
            'fontname': 'Arial'
        })
    
    # 设置其他文本元素的样式（如特征名称等）
    for text in ax.texts:
        text.set_fontweight('bold')
        text.set_fontname('Arial')
        text.set_fontsize(14)

    # 调整边距
    plt.subplots_adjust(left=0.4, right=0.9)

    # 设置坐标轴样式
    ax.spines['top'].set_visible(False)
    ax.spines['top'].set_color('black')
    ax.spines['top'].set_linewidth(2)
    ax.spines['right'].set_visible(False)
    ax.spines['right'].set_color('black')
    ax.spines['right'].set_linewidth(2)
    ax.spines['left'].set_visible(False)
    ax.spines['left'].set_color('black')
    ax.spines['left'].set_linewidth(2)
    ax.spines['bottom'].set_visible(False)
    ax.spines['bottom'].set_color('black')
    ax.spines['bottom'].set_linewidth(3)

    plot_path = os.path.join(output_dir, "waterfall_plots", f"shap_waterfall_{sample_index}.png")
    plt.savefig(plot_path, dpi=300, bbox_inches='tight', transparent=True)
    plt.close()
    return plot_path

def main():
    os.makedirs(output_dir, exist_ok=True)
    
    print("Loading data and model...")
    X, y, model = load_data_and_model()
    
    print(f"Preparing data for SHAP analysis (first {sample_size} samples)...")
    X_shap = prepare_shap_data(X, y)
    
    print("Calculating SHAP values...")
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X_shap)
    
    print("Generating SHAP plots...")
    summary_path = plot_shap_summary(shap_values, X_shap, output_dir)
    bar_path = plot_shap_bar(shap_values, X_shap, output_dir)
    dependence_paths = plot_shap_dependence(shap_values, X_shap, output_dir)
    
    print("Generating Waterfall plots...")
    waterfall_paths = []
    # 确保选择的样本索引在新的、更小的数据集范围内
    for i in [0, min(5, len(X_shap)-1), min(188, len(X_shap)-1)]:
        try:
            path = plot_shap_waterfall(explainer, shap_values, X_shap, i, output_dir)
            waterfall_paths.append(path)
        except Exception as e:
            print(f"Error generating waterfall plot for sample {i}: {str(e)}")
    
    # 保存SHAP值
    pd.DataFrame(shap_values, columns=X_shap.columns).to_csv(
        os.path.join(output_dir, "shap_values.csv"), index=False)
    
    print("\nSHAP analysis completed!")
    print(f"Results saved to: {output_dir}")
    print(f"Analysis was performed on the first {len(X_shap)} samples.")

if __name__ == "__main__":
    main()

Loading data and model...
Preparing data for SHAP analysis (first 200 samples)...
Calculating SHAP values...
Generating SHAP plots...
Generating Waterfall plots...

SHAP analysis completed!
Results saved to: shap_analysis
Analysis was performed on the first 200 samples.


In [44]:
# -*- coding: utf-8 -*-
import shap
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import os
import numpy as np
from matplotlib import rcParams

# ==================== Parameter Configuration ====================
MODEL_PATH = "xgboost_model_ncv/xgboost_model.pkl"  # Model path
DATA_PATH = "归一化数据-20251120 -去除前两列.xlsx"                  # Data path
OUTPUT_DIR = "shap_force_analysis"                  # Output directory
TARGET_COL = "SO2 tolerance"                       # Target column name
SAMPLE_INDICES = [0, 5, 188]                       # Sample indices to analyze
DPI = 300                                          # Output resolution
CONTRIBUTION_THRESHOLD = 0.1
FONT = {'family': 'Arial', 'weight': 'bold', 'size': 10}  # Font settings

# ==================== Initialization ====================
# Create output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Set global font settings
rcParams['font.family'] = FONT['family']
rcParams['font.weight'] = FONT['weight']
rcParams['font.size'] = FONT['size']
rcParams['axes.labelweight'] = 'bold'
rcParams['axes.titleweight'] = 'bold'

# ==================== Load Model and Data ====================
# Load trained model
model = joblib.load(MODEL_PATH)

# Read data
data = pd.read_excel(DATA_PATH)
X = data.drop(columns=[TARGET_COL])  # Feature data
y = data[TARGET_COL]  # Target variable

# ==================== SHAP Analysis ====================
# Create SHAP explainer
explainer = shap.Explainer(model)
shap_values = explainer(X)

# ==================== Generate and Save Force Plots ====================
for idx in SAMPLE_INDICES:
    if idx >= len(X):
        print(f"Warning: Sample index {idx} out of range, skipping...")
        continue
    
    # Create figure with specified size
    plt.figure(figsize=(10, 5))
    
    # Format feature names to display three decimal places
    feature_names = X.columns.tolist()
    formatted_features = []
    for i, feat in enumerate(feature_names):
        val = X.iloc[idx, i]
        formatted_val = f"{val:.3f}"  # Format to three decimal places
        formatted_features.append(f"{feat} = {formatted_val}")
    
    # Generate force plot with formatted feature names
    force_plot = shap.plots.force(
        explainer.expected_value,
        shap_values.values[idx, :],
        features=X.iloc[idx, :],
        feature_names=formatted_features,  # Use formatted feature names
        matplotlib=True,
        show=False
    )
    
    # Customize plot appearance
    plt.title(f"SHAP Force Plot - Sample {idx}", fontweight='bold', pad=20)
    plt.xlabel("Feature value impact on model output", fontweight='bold')
    
    # Adjust layout
    plt.tight_layout()
    
    # Save figure
    output_path = os.path.join(OUTPUT_DIR, f"force_plot_sample_{idx}.png")
    plt.savefig(output_path, dpi=DPI, bbox_inches='tight')
    plt.close()
    
    print(f"Saved force plot for sample {idx} to: {output_path}")

print("All SHAP force plots generated successfully!")    

Saved force plot for sample 0 to: shap_force_analysis\force_plot_sample_0.png
Saved force plot for sample 5 to: shap_force_analysis\force_plot_sample_5.png
Saved force plot for sample 188 to: shap_force_analysis\force_plot_sample_188.png
All SHAP force plots generated successfully!


In [28]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
import os
import joblib

# ==================== 参数配置 ====================
model_path = "xgboost_model_ncv/xgboost_model.pkl"
input_path = "归一化数据-20251120.xlsx"
output_dir = "ice_analysis_results"
target_column = "SO2 tolerance"

# 图表样式参数
figsize = (17, 14)
title_font = {'family': 'Arial', 'size': 18, 'weight': 'bold'}
label_font = {'family': 'Arial', 'size': 40, 'weight': 'bold'}
tick_font = {'family': 'Arial', 'size': 36, 'weight': 'bold'}
legend_font = {'family': 'Arial', 'size': 36, 'weight': 'bold'}

line_alpha = 0.3
line_color = '#80AAF3'
line_width = 3

avg_line_color = '#d62728'
avg_line_width = 3

border_width = 4
border_color = 'black'

tick_width = 3
tick_length = 8

dpi = 300

# ==================== 函数定义 ====================
def load_data_and_model():
    """加载数据和模型"""
    df = pd.read_excel(input_path)
    if target_column not in df.columns:
        raise ValueError(f"目标列 {target_column} 不存在于数据中")
    
    X = df.drop(columns=[target_column]).select_dtypes(include=['number'])
    model = joblib.load(model_path)
    return X, model

def plot_ice_analysis():
    """执行ICE分析并绘制图表"""
    os.makedirs(output_dir, exist_ok=True)
    X, model = load_data_and_model()
    
    for feature in X.columns:
        fig = plt.figure(figsize=figsize, facecolor='white')
        ax = plt.gca()
        
        # 设置背景为白色（移除透明设置）
        fig.patch.set_facecolor('white')
        ax.set_facecolor('white')
        
        grid_values = np.linspace(X[feature].min(), X[feature].max(), 100)
        ice_lines = []
        
        for _, sample in X.iterrows():
            X_ice = pd.DataFrame([sample.values]*len(grid_values), columns=X.columns)
            X_ice[feature] = grid_values
            
            preds = model.predict(xgb.DMatrix(X_ice))
            ice_lines.append(preds)
            plt.plot(grid_values, preds, color=line_color, alpha=line_alpha, linewidth=line_width)
        
        # 绘制平均线
        plt.plot(grid_values, np.mean(ice_lines, axis=0), 
                color=avg_line_color, linewidth=avg_line_width, label='Average')
        
        # 设置标题和标签
        plt.title(f'ICE Plot for {feature}', fontdict=title_font, pad=20)
        plt.xlabel(feature, fontdict=label_font, labelpad=10)
        plt.ylabel('Predicted value', fontdict=label_font, labelpad=10)
        
        # 设置刻度
        ax.tick_params(axis='both', which='major', 
                      labelsize=tick_font['size'],
                      width=tick_width,
                      length=tick_length)
        
        # 设置刻度标签字体
        for label in ax.get_xticklabels():
            label.set_fontproperties(tick_font)
        for label in ax.get_yticklabels():
            label.set_fontproperties(tick_font)
        
        # 设置图例
        legend = plt.legend(prop=legend_font, frameon=False, loc='upper right')
        for text in legend.get_texts():
            text.set_weight('bold')
        
        # 设置边框
        for spine in ax.spines.values():
            spine.set_linewidth(border_width)
            spine.set_color(border_color)
        
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"ice_{feature}.png"), 
                   dpi=dpi, bbox_inches='tight')  # 移除transparent=True
        plt.close()

# ==================== 主程序 ====================
if __name__ == "__main__":
    print("开始ICE分析...")
    plot_ice_analysis()
    print(f"分析完成，结果保存在: {output_dir}")

开始ICE分析...
分析完成，结果保存在: ice_analysis_results


In [21]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import xgboost as xgb
import joblib
import os
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# ==================== 参数配置 ====================
model_path = "xgboost_model_ncv/xgboost_model.pkl"
input_path = "归一化数据-20251120.xlsx"
output_dir = "ice_smooth_results"
target_column = "SO2 tolerance"
feature_to_plot = "EN."

# 感兴趣的区间（绘图和拟合都只在这个范围内）
focus_range = (0.7, 0.8)
# 自定义Y轴范围
ylim_range = (0.7, 0.9)

# 拟合与统计参数
poly_degree = 5  # 多项式拟合阶数
confidence_level = 1.96  # 仍需计算置信区间，这里保持95%

# 图表样式
figsize = (10, 6)
title_font = {'family': 'Arial', 'size': 22, 'weight': 'bold'}
label_font = {'family': 'Arial', 'size': 20, 'weight': 'bold'}
tick_font = {'family': 'Arial', 'size': 18}

# 线条和颜色样式
raw_curve_style = {'color': 'gray', 'linestyle': '--', 'linewidth': 2, 'label': '原始平均曲线'}
fit_curve_style = {'color': 'red', 'linestyle': '-', 'linewidth': 3, 'label': f'{poly_degree}阶多项式拟合'}

# ==================== 函数定义 ====================
def load_data_and_model():
    """加载数据和模型"""
    df = pd.read_excel(input_path)
    missing_cols = [col for col in [target_column, feature_to_plot] if col not in df.columns]
    if missing_cols:
        raise ValueError(f"缺失列: {', '.join(missing_cols)}")
    
    X = df.drop(columns=[target_column]).select_dtypes(include=['number'])
    model = joblib.load(model_path)
    return X, model

def polynomial_func(x, *coefficients):
    """多项式函数，用于曲线拟合"""
    return sum(c * x**i for i, c in enumerate(coefficients))

def calculate_and_fit_data():
    """计算指定区间内的ICE数据（含置信区间）并进行拟合"""
    X, model = load_data_and_model()
    
    grid_values = np.linspace(focus_range[0] - 0.01, focus_range[1] + 0.01, 500)
    all_predictions = []

    print(f"正在计算每个样本的ICE曲线...")
    for idx, (_, sample) in enumerate(X.iterrows()):
        X_ice = pd.DataFrame([sample.values] * len(grid_values), columns=X.columns)
        X_ice[feature_to_plot] = grid_values
        preds = model.predict(xgb.DMatrix(X_ice))
        all_predictions.append(preds)
        
        if (idx + 1) % 50 == 0:
            print(f"已处理 {idx + 1} 个样本...")

    predictions_array = np.array(all_predictions)
    mean_preds = predictions_array.mean(axis=0)
    std_preds = predictions_array.std(axis=0)  # 仍需计算标准差
    
    # 计算置信区间上下界（用于导出）
    lower_bound = mean_preds - confidence_level * std_preds
    upper_bound = mean_preds + confidence_level * std_preds
    
    full_df = pd.DataFrame({
        feature_to_plot: grid_values,
        'Mean_Pred': mean_preds,
        'Std_Pred': std_preds,
        'Lower_Bound': lower_bound,
        'Upper_Bound': upper_bound
    })
    
    mask = (full_df[feature_to_plot] >= focus_range[0]) & (full_df[feature_to_plot] <= focus_range[1])
    focused_df = full_df[mask].copy().reset_index(drop=True)
    
    print(f"在区间 {focus_range} 内找到 {len(focused_df)} 个数据点，准备进行拟合...")

    if focused_df.empty:
        raise ValueError(f"在指定的区间 {focus_range} 内没有找到任何数据点。")

    x_for_fit = focused_df[feature_to_plot].values
    y_for_fit = focused_df['Mean_Pred'].values
    
    initial_guess = [1] * (poly_degree + 1)
    params, _ = curve_fit(
        polynomial_func, 
        x_for_fit, 
        y_for_fit,
        p0=initial_guess,
        maxfev=10000
    )
    
    x_fit_smooth = np.linspace(focus_range[0], focus_range[1], 300)
    y_fit_smooth = polynomial_func(x_fit_smooth, *params)
    
    return focused_df, x_fit_smooth, y_fit_smooth

def plot_and_export_results(focused_df, x_fit_smooth, y_fit_smooth):
    """绘制并导出指定区间内的结果（图中不显示置信区间）"""
    os.makedirs(output_dir, exist_ok=True)
    
    # 导出数据到Excel（包含置信区间数据）
    output_filename = f"ice_focused_on_{focus_range[0]}_{focus_range[1]}_{feature_to_plot.replace('.', '_')}_with_ci_data.xlsx"
    output_path = os.path.join(output_dir, output_filename)
    
    export_df = focused_df[[feature_to_plot, 'Mean_Pred', 'Std_Pred', 'Lower_Bound', 'Upper_Bound']].copy()
    export_df.columns = [feature_to_plot, '原始平均预测值', '预测值标准差', '95%置信区间下界', '95%置信区间上界']
    
    fit_curve_df = pd.DataFrame({
        feature_to_plot: x_fit_smooth,
        '拟合曲线预测值': y_fit_smooth
    })
    
    with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
        export_df.to_excel(writer, sheet_name='区间内原始数据(含置信区间)', index=False)
        fit_curve_df.to_excel(writer, sheet_name='拟合曲线数据', index=False)
        
    print(f"数据已导出至: {output_path}")
    
    # 绘制图表（不包含置信区间）
    fig, ax = plt.subplots(figsize=figsize)
    
    # 绘制原始平均曲线（灰色虚线）
    ax.plot(focused_df[feature_to_plot], focused_df['Mean_Pred'], **raw_curve_style)
    
    # 绘制拟合曲线（红色实线）
    ax.plot(x_fit_smooth, y_fit_smooth, **fit_curve_style)
    
    # 精确设置坐标轴范围
    ax.set_xlim(focus_range)
    ax.set_ylim(ylim_range)
    
    # 设置图表标题和标签
    ax.set_title(f'ICE分析: {feature_to_plot} (聚焦区间: {focus_range[0]} - {focus_range[1]})', fontdict=title_font)
    ax.set_xlabel(feature_to_plot, fontdict=label_font)
    ax.set_ylabel('预测值', fontdict=label_font)
    
    # 设置刻度字体
    ax.tick_params(axis='both', which='major', labelsize=tick_font['size'])
    for label in ax.get_xticklabels():
        label.set_fontproperties(tick_font)
    for label in ax.get_yticklabels():
        label.set_fontproperties(tick_font)
    
    # 设置图例
    ax.legend(prop=tick_font, frameon=False, fancybox=False, shadow=False)
    
    # 添加网格线
    ax.grid(False, linestyle=':', alpha=0.6)
    
    # 保存图表
    plot_filename = f"ice_focused_on_{focus_range[0]}_{focus_range[1]}_{feature_to_plot.replace('.', '_')}_no_ci_plot.png"
    plot_path = os.path.join(output_dir, plot_filename)
    plt.tight_layout()
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    print(f"图表已保存至: {plot_path}")
    plt.close()

# ==================== 主程序 ====================
if __name__ == "__main__":
    print(f"开始生成 '{feature_to_plot}' 在区间 {focus_range} 内的ICE分析图 (图中不显示置信区间)...")
    try:
        focused_df, x_fit_smooth, y_fit_smooth = calculate_and_fit_data()
        plot_and_export_results(focused_df, x_fit_smooth, y_fit_smooth)
    except Exception as e:
        print(f"错误: {str(e)}")

开始生成 'EN.' 在区间 (0.7, 0.8) 内的ICE分析图 (图中不显示置信区间)...
正在计算每个样本的ICE曲线...
已处理 50 个样本...
已处理 100 个样本...
已处理 150 个样本...
已处理 200 个样本...
在区间 (0.7, 0.8) 内找到 416 个数据点，准备进行拟合...
数据已导出至: ice_smooth_results\ice_focused_on_0.7_0.8_EN__with_ci_data.xlsx


C:\Users\miaoxinyi\AppData\Local\Temp\ipykernel_30288\4047348588.py:161: UserWarning: First parameter to grid() is false, but line properties are supplied. The grid will be enabled.
  ax.grid(False, linestyle=':', alpha=0.6)
C:\Users\miaoxinyi\AppData\Local\Temp\ipykernel_30288\4047348588.py:166: UserWarning: Glyph 39044 (\N{CJK UNIFIED IDEOGRAPH-9884}) missing from font(s) Arial.
  plt.tight_layout()
C:\Users\miaoxinyi\AppData\Local\Temp\ipykernel_30288\4047348588.py:166: UserWarning: Glyph 27979 (\N{CJK UNIFIED IDEOGRAPH-6D4B}) missing from font(s) Arial.
  plt.tight_layout()
C:\Users\miaoxinyi\AppData\Local\Temp\ipykernel_30288\4047348588.py:166: UserWarning: Glyph 20540 (\N{CJK UNIFIED IDEOGRAPH-503C}) missing from font(s) Arial.
  plt.tight_layout()
C:\Users\miaoxinyi\AppData\Local\Temp\ipykernel_30288\4047348588.py:166: UserWarning: Glyph 20998 (\N{CJK UNIFIED IDEOGRAPH-5206}) missing from font(s) Arial.
  plt.tight_layout()
C:\Users\miaoxinyi\AppData\Local\Temp\ipykernel_30288\4

图表已保存至: ice_smooth_results\ice_focused_on_0.7_0.8_EN__no_ci_plot.png


In [8]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import xgboost as xgb
import joblib
import os
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# ==================== 参数配置 ====================
model_path = "xgboost_model_ncv/xgboost_model.pkl"
input_path = "归一化数据-20251120.xlsx"
output_dir = "ice_smooth_results"
target_column = "SO2 tolerance"
feature_to_plot = "EN."

# 感兴趣的区间（绘图和拟合都只在这个范围内）
focus_range = (0.7, 0.8)

# 拟合参数
poly_degree = 5  # 多项式拟合阶数
confidence_level = 1.96  # 95%置信区间的系数

# 图表样式
figsize = (10, 6)
title_font = {'family': 'Arial', 'size': 16, 'weight': 'bold'}
label_font = {'family': 'Arial', 'size': 14, 'weight': 'bold'}
tick_font = {'family': 'Arial', 'size': 12}

# 线条和颜色样式
raw_curve_style = {'color': 'gray', 'linestyle': '--', 'linewidth': 2, 'label': '原始平均曲线'}
fit_curve_style = {'color': 'red', 'linestyle': '-', 'linewidth': 3, 'label': f'{poly_degree}阶多项式拟合'}
confidence_interval_style = {'color': 'gray', 'alpha': 0.3, 'label': '95% 置信区间'}

# ==================== 函数定义 ====================
def load_data_and_model():
    """加载数据和模型"""
    df = pd.read_excel(input_path)
    missing_cols = [col for col in [target_column, feature_to_plot] if col not in df.columns]
    if missing_cols:
        raise ValueError(f"缺失列: {', '.join(missing_cols)}")
    
    X = df.drop(columns=[target_column]).select_dtypes(include=['number'])
    model = joblib.load(model_path)
    return X, model

def polynomial_func(x, *coefficients):
    """多项式函数，用于曲线拟合"""
    return sum(c * x**i for i, c in enumerate(coefficients))

def calculate_and_fit_data():
    """计算指定区间内的ICE数据并进行拟合"""
    X, model = load_data_and_model()
    
    # 为了获得更精细的结果，在稍宽的范围内生成网格点，之后再精确筛选
    grid_values = np.linspace(focus_range[0] - 0.01, focus_range[1] + 0.01, 500)
    all_predictions = []

    print(f"正在计算每个样本的ICE曲线...")
    for idx, (_, sample) in enumerate(X.iterrows()):
        X_ice = pd.DataFrame([sample.values] * len(grid_values), columns=X.columns)
        X_ice[feature_to_plot] = grid_values
        preds = model.predict(xgb.DMatrix(X_ice))
        all_predictions.append(preds)
        
        if (idx + 1) % 50 == 0:
            print(f"已处理 {idx + 1} 个样本...")

    # 计算平均曲线和标准差
    predictions_array = np.array(all_predictions)
    mean_preds = predictions_array.mean(axis=0)
    std_preds = predictions_array.std(axis=0)
    
    # 创建DataFrame并精确筛选出我们关注的区间
    full_df = pd.DataFrame({
        feature_to_plot: grid_values,
        'Mean_Pred': mean_preds,
        'Std_Pred': std_preds
    })
    mask = (full_df[feature_to_plot] >= focus_range[0]) & (full_df[feature_to_plot] <= focus_range[1])
    focused_df = full_df[mask].copy().reset_index(drop=True)
    
    print(f"在区间 {focus_range} 内找到 {len(focused_df)} 个数据点，准备进行拟合...")

    if focused_df.empty:
        raise ValueError(f"在指定的区间 {focus_range} 内没有找到任何数据点。")

    # 使用筛选后的数据进行多项式拟合
    x_for_fit = focused_df[feature_to_plot].values
    y_for_fit = focused_df['Mean_Pred'].values
    
    initial_guess = [1] * (poly_degree + 1)
    params, _ = curve_fit(
        polynomial_func, 
        x_for_fit, 
        y_for_fit,
        p0=initial_guess,
        maxfev=10000
    )
    
    # 为了使拟合曲线看起来更平滑，在关注区间内生成更密集的点来绘制拟合线
    x_fit_smooth = np.linspace(focus_range[0], focus_range[1], 300)
    y_fit_smooth = polynomial_func(x_fit_smooth, *params)
    
    return focused_df, x_fit_smooth, y_fit_smooth

def plot_and_export_results(focused_df, x_fit_smooth, y_fit_smooth):
    """绘制并导出指定区间内的结果"""
    os.makedirs(output_dir, exist_ok=True)
    
    # 导出数据到Excel
    output_filename = f"ice_focused_on_{focus_range[0]}_{focus_range[1]}_{feature_to_plot.replace('.', '_')}.xlsx"
    output_path = os.path.join(output_dir, output_filename)
    
    # 准备导出数据，包括原始平均点和拟合后的平滑点
    export_df = focused_df[[feature_to_plot, 'Mean_Pred', 'Std_Pred']].copy()
    export_df.columns = [feature_to_plot, '原始平均预测值', '预测值标准差']
    
    # 添加拟合曲线的数据
    fit_curve_df = pd.DataFrame({
        feature_to_plot: x_fit_smooth,
        '原始平均预测值': np.nan,
        '预测值标准差': np.nan
    })
    fit_curve_df['拟合曲线预测值'] = y_fit_smooth
    
    # 使用ExcelWriter将数据写入不同的工作表
    with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
        export_df.to_excel(writer, sheet_name='区间内原始数据', index=False)
        fit_curve_df.to_excel(writer, sheet_name='拟合曲线数据', index=False)
        
    print(f"数据已导出至: {output_path}")
    
    # 绘制图表
    fig, ax = plt.subplots(figsize=figsize)
    
    # 绘制置信区间
    ax.fill_between(
        focused_df[feature_to_plot],
        focused_df['Mean_Pred'] - confidence_level * focused_df['Std_Pred'],
        focused_df['Mean_Pred'] + confidence_level * focused_df['Std_Pred'],
        **confidence_interval_style
    )
    
    # 绘制原始平均曲线（灰色虚线）
    ax.plot(focused_df[feature_to_plot], focused_df['Mean_Pred'], **raw_curve_style)
    
    # 绘制拟合曲线（红色实线）
    ax.plot(x_fit_smooth, y_fit_smooth, **fit_curve_style)
    
    # 精确设置坐标轴范围，只显示我们关注的区间
    ax.set_xlim(focus_range)
    # y轴范围可以自动调整，也可以手动设置以突出变化
    # ax.set_ylim([y_min, y_max]) 
    
    # 设置图表标题和标签
    ax.set_title(f'ICE分析: {feature_to_plot} (聚焦区间: {focus_range[0]} - {focus_range[1]})', fontdict=title_font)
    ax.set_xlabel(feature_to_plot, fontdict=label_font)
    ax.set_ylabel('预测值', fontdict=label_font)
    
    # 设置刻度字体
    ax.tick_params(axis='both', which='major', labelsize=tick_font['size'])
    for label in ax.get_xticklabels():
        label.set_fontproperties(tick_font)
    for label in ax.get_yticklabels():
        label.set_fontproperties(tick_font)
    
    # 设置图例
    ax.legend(prop=tick_font, frameon=True, fancybox=True, shadow=True)
    
    # 添加网格线
    ax.grid(True, linestyle=':', alpha=0.6)
    
    # 保存图表
    plot_filename = f"ice_focused_on_{focus_range[0]}_{focus_range[1]}_{feature_to_plot.replace('.', '_')}.png"
    plot_path = os.path.join(output_dir, plot_filename)
    plt.tight_layout()
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    print(f"图表已保存至: {plot_path}")
    plt.close()

# ==================== 主程序 ====================
if __name__ == "__main__":
    print(f"开始生成 '{feature_to_plot}' 在区间 {focus_range} 内的ICE分析图...")
    try:
        focused_df, x_fit_smooth, y_fit_smooth = calculate_and_fit_data()
        plot_and_export_results(focused_df, x_fit_smooth, y_fit_smooth)
    except Exception as e:
        print(f"错误: {str(e)}")

开始生成 'EN.' 在区间 (0.7, 0.8) 内的ICE分析图...
正在计算每个样本的ICE曲线...
已处理 50 个样本...
已处理 100 个样本...
已处理 150 个样本...
已处理 200 个样本...
在区间 (0.7, 0.8) 内找到 416 个数据点，准备进行拟合...
数据已导出至: ice_smooth_results\ice_focused_on_0.7_0.8_EN_.xlsx


C:\Users\miaoxinyi\AppData\Local\Temp\ipykernel_30288\3244283329.py:177: UserWarning: Glyph 39044 (\N{CJK UNIFIED IDEOGRAPH-9884}) missing from font(s) Arial.
  plt.tight_layout()
C:\Users\miaoxinyi\AppData\Local\Temp\ipykernel_30288\3244283329.py:177: UserWarning: Glyph 27979 (\N{CJK UNIFIED IDEOGRAPH-6D4B}) missing from font(s) Arial.
  plt.tight_layout()
C:\Users\miaoxinyi\AppData\Local\Temp\ipykernel_30288\3244283329.py:177: UserWarning: Glyph 20540 (\N{CJK UNIFIED IDEOGRAPH-503C}) missing from font(s) Arial.
  plt.tight_layout()
C:\Users\miaoxinyi\AppData\Local\Temp\ipykernel_30288\3244283329.py:177: UserWarning: Glyph 20998 (\N{CJK UNIFIED IDEOGRAPH-5206}) missing from font(s) Arial.
  plt.tight_layout()
C:\Users\miaoxinyi\AppData\Local\Temp\ipykernel_30288\3244283329.py:177: UserWarning: Glyph 26512 (\N{CJK UNIFIED IDEOGRAPH-6790}) missing from font(s) Arial.
  plt.tight_layout()
C:\Users\miaoxinyi\AppData\Local\Temp\ipykernel_30288\3244283329.py:177: UserWarning: Glyph 32858 (\

图表已保存至: ice_smooth_results\ice_focused_on_0.7_0.8_EN_.png


In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import os
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.ticker import MaxNLocator

# 参数配置
model_path = "xgboost_model_ncv/xgboost_model.pkl"  # 模型路径
input_path = "归一化数据-20251120.xlsx"  # 输入数据路径
output_dir = "pdp_analysis"  # 输出目录
feature1 = "EN."  # 第一个特征
feature2 = "I.R."  # 第二个特征
random_state = 42  # 随机种子

# 创建输出目录
os.makedirs(output_dir, exist_ok=True)

def prepare_data(input_path):
    """数据加载与预处理"""
    df = pd.read_excel(input_path)
    return df

def load_model(model_path):
    """加载训练好的XGBoost模型"""
    return joblib.load(model_path)

def calculate_pdp_2d(model, df, feature1, feature2, num_points=20):
    """计算两个特征的2D部分依赖"""
    # 确保特征存在
    if feature1 not in df.columns or feature2 not in df.columns:
        raise ValueError(f"特征 {feature1} 或 {feature2} 不存在于数据中")
    
    # 保存原始特征值
    original_features = df[[feature1, feature2]].copy()
    
    # 生成特征网格
    feature1_values = np.linspace(df[feature1].min(), df[feature1].max(), num_points)
    feature2_values = np.linspace(df[feature2].min(), df[feature2].max(), num_points)
    feature1_mesh, feature2_mesh = np.meshgrid(feature1_values, feature2_values)
    
    # 初始化预测结果数组
    pdp_values = np.zeros((num_points, num_points))
    
    # 对每个特征组合计算预测值
    for i, f1_val in enumerate(feature1_values):
        for j, f2_val in enumerate(feature2_values):
            # 创建临时数据副本
            temp_df = df.copy()
            temp_df[feature1] = f1_val
            temp_df[feature2] = f2_val
            
            # 准备特征矩阵
            X = temp_df.drop(columns=[col for col in temp_df.columns 
                                     if col not in model.feature_names])
            
            # 进行预测
            dmatrix = xgb.DMatrix(X)
            predictions = model.predict(dmatrix)
            
            # 计算平均预测值作为PDP值
            pdp_values[j, i] = np.mean(predictions)
    
    # 新增：返回原始的一维特征值数组
    return feature1_mesh, feature2_mesh, pdp_values, feature1_values, feature2_values

def plot_pdp_2d(feature1, feature2, feature1_mesh, feature2_mesh, pdp_values, output_dir):
    """绘制2D部分依赖图"""
    # 设置全局字体样式
    plt.rcParams['font.family'] = 'Arial'
    plt.rcParams['font.weight'] = 'bold'
    
    # 创建自定义颜色映射
    custom_cmap = LinearSegmentedColormap.from_list(
        'blue_to_red', ['#1f77b4', '#ffffff', '#d62728'], N=256)
    
    # 创建图形
    fig, ax = plt.subplots(figsize=(14, 12), facecolor='white')
    
    # 绘制热力图
    im = ax.pcolormesh(
        feature1_mesh, feature2_mesh, pdp_values,
        cmap=custom_cmap, shading='auto',
        vmin=pdp_values.min(), vmax=pdp_values.max()
    )
    
    # 添加颜色条
    cbar = fig.colorbar(im, ax=ax, pad=0.02)
    cbar.set_label('Predicted Value', fontsize=24, fontname='Arial', fontweight='bold')
    cbar.ax.tick_params(labelsize=20, direction='out', length=6, width=2)
    
    # 设置坐标轴标签和标题
    ax.set_xlabel(feature1, fontsize=28, fontname='Arial', fontweight='bold', labelpad=10)
    ax.set_ylabel(feature2, fontsize=28, fontname='Arial', fontweight='bold', labelpad=10)
    ax.set_title(f'2D Partial Dependence Plot for {feature1} and {feature2}',
                fontsize=32, fontname='Arial', fontweight='bold', pad=20)
    
    # 设置坐标轴刻度
    ax.xaxis.set_major_locator(MaxNLocator(nbins=6, integer=False))
    ax.yaxis.set_major_locator(MaxNLocator(nbins=6, integer=False))
    
    # 设置刻度标签样式
    for tick in ax.get_xticklabels() + ax.get_yticklabels():
        tick.set_fontname('Arial')
        tick.set_fontsize(20)
        tick.set_weight('bold')
    
    # 设置边框样式
    for spine in ax.spines.values():
        spine.set_linewidth(2.5)
        spine.set_color('black')
    
    # 添加网格线
    ax.grid(True, linestyle='--', alpha=0.7, linewidth=1.5)
    
    # 调整布局
    plt.tight_layout()
    
    # 保存图形
    plot_path = os.path.join(output_dir, f"pdp_2d_{feature1}_{feature2}.png")
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    return plot_path

def plot_contour_pdp_2d(feature1, feature2, feature1_mesh, feature2_mesh, pdp_values, output_dir):
    """绘制2D部分依赖的等高线图（优化标签堆叠）"""
    # 设置全局字体样式
    plt.rcParams['font.family'] = 'Arial'
    plt.rcParams['font.weight'] = 'bold'
    
    # 创建自定义颜色映射
    custom_cmap = LinearSegmentedColormap.from_list(
        'blue_to_red', ['#1f77b4', '#ffffff', '#d62728'], N=256)
    
    # 创建图形
    fig, ax = plt.subplots(figsize=(14, 12), facecolor='white')
    
    # 绘制等高线图
    contour = ax.contourf(
        feature1_mesh, feature2_mesh, pdp_values,
        cmap=custom_cmap, levels=15,
        vmin=pdp_values.min(), vmax=pdp_values.max()
    )
    
    # 添加颜色条
    cbar = fig.colorbar(contour, ax=ax, pad=0.02)
    cbar.set_label('Predicted Value', fontsize=24, fontname='Arial', fontweight='bold')
    cbar.ax.tick_params(labelsize=20, direction='out', length=6, width=2)
    
    # 添加等高线（仅绘制关键等高线，减少密集标注）
    contour_lines = ax.contour(
        feature1_mesh, feature2_mesh, pdp_values,
        colors='black', linewidths=1.5, levels=8  # 减少 levels 数量，控制标注密度
    )
    
    # 优化：仅在稀疏区域标注，避免堆叠
    ax.clabel(contour_lines, inline=True, fontsize=16, levels=contour_lines.levels[::2])
    
    # 设置坐标轴标签和标题
    ax.set_xlabel(feature1, fontsize=28, fontname='Arial', fontweight='bold', labelpad=10)
    ax.set_ylabel(feature2, fontsize=28, fontname='Arial', fontweight='bold', labelpad=10)
    # 修改：标题中去掉了“(Marked Key Points)”
    ax.set_title(f'Contour Plot of 2D Partial Dependence for {feature1} and {feature2}',
                fontsize=32, fontname='Arial', fontweight='bold', pad=20)
    
    # 设置坐标轴刻度
    ax.xaxis.set_major_locator(MaxNLocator(nbins=6, integer=False))
    ax.yaxis.set_major_locator(MaxNLocator(nbins=6, integer=False))
    
    # 设置刻度标签样式
    for tick in ax.get_xticklabels() + ax.get_yticklabels():
        tick.set_fontname('Arial')
        tick.set_fontsize(20)
        tick.set_weight('bold')
    
    # 设置边框样式
    for spine in ax.spines.values():
        spine.set_linewidth(2.5)
        spine.set_color('black')
    
    # 添加网格线
    ax.grid(True, linestyle='--', alpha=0.7, linewidth=1.5)
    
    # 调整布局
    plt.tight_layout()
    
    # 保存图形
    plot_path = os.path.join(output_dir, f"contour_pdp_2d_{feature1}_{feature2}.png")
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    return plot_path

def main():
    print(f"开始{feature1}和{feature2}的2D部分依赖分析...")
    
    # 1. 加载数据和模型
    print("加载数据和模型...")
    df = prepare_data(input_path)
    model = load_model(model_path)
    
    # 2. 计算2D部分依赖
    print("计算2D部分依赖...")
    # 修改：接收 feature1_values 和 feature2_values
    feature1_mesh, feature2_mesh, pdp_values, feature1_values, feature2_values = calculate_pdp_2d(
        model, df, feature1, feature2, num_points=25)
    
    # 3. 绘制2D PDP热力图
    print("绘制2D PDP热力图...")
    heatmap_path = plot_pdp_2d(
        feature1, feature2, feature1_mesh, feature2_mesh, pdp_values, output_dir)
    print(f"2D PDP热力图已保存至：{heatmap_path}")
    
    # 4. 绘制2D PDP等高线图
    print("绘制2D PDP等高线图...")
    # 修改：调用时不再需要传递 feature1_values 和 feature2_values
    contour_path = plot_contour_pdp_2d(
        feature1, feature2, feature1_mesh, feature2_mesh, pdp_values, output_dir)
    print(f"2D PDP等高线图已保存至：{contour_path}")
    
    # 5. 保存PDP数据
    print("保存PDP分析数据...")
    pdp_data = {
        'feature1_values': feature1_mesh.flatten(),
        'feature2_values': feature2_mesh.flatten(),
        'pdp_values': pdp_values.flatten()
    }
    pdp_df = pd.DataFrame(pdp_data)
    pdp_df.to_csv(os.path.join(output_dir, f"pdp_2d_{feature1}_{feature2}.csv"), index=False)
    print(f"PDP数据已保存至：{os.path.join(output_dir, f'pdp_2d_{feature1}_{feature2}.csv')}")
    
    print(f"\n{feature1}和{feature2}的2D部分依赖分析完成！所有结果保存在目录：{output_dir}")

if __name__ == "__main__":
    main()

开始EN.和I.R.的2D部分依赖分析...
加载数据和模型...
计算2D部分依赖...
绘制2D PDP热力图...
2D PDP热力图已保存至：pdp_analysis\pdp_2d_EN._I.R..png
绘制2D PDP等高线图...
2D PDP等高线图已保存至：pdp_analysis\contour_pdp_2d_EN._I.R..png
保存PDP分析数据...
PDP数据已保存至：pdp_analysis\pdp_2d_EN._I.R..csv

EN.和I.R.的2D部分依赖分析完成！所有结果保存在目录：pdp_analysis


In [30]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import os
import joblib

# ==================== 参数配置 ====================
# 请确保这些参数与你原始训练脚本中的参数一致
input_path = "归一化数据-20251120.xlsx"
target_column = "SO2 tolerance"
outer_test_size = 0.2
random_state = 42
model_params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'eta': 0.1,
    'max_depth': 6,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'seed': random_state,
    'n_jobs': -1
}

# 指定要依次移除的特征列表
features_to_ablate = ['EN.', 'I.R.', 'I.E.', 'Density', 'M.P.', 'T.C.']

# 输出目录
output_dir = "xgboost_ablation_analysis"

# ==================== 函数定义 (复用自原脚本) ====================
def prepare_data(input_path, target_column, features_to_drop=None):
    """数据加载与预处理"""
    df = pd.read_excel(input_path)
    if target_column not in df.columns:
        raise ValueError(f"目标列 {target_column} 不存在于数据中")
    
    y = df[target_column]
    X = df.drop(columns=[target_column])

    # 如果指定了要删除的特征，则在数据预处理阶段就移除它们
    if features_to_drop:
        # 确保要删除的特征确实存在
        features_to_drop = [f for f in features_to_drop if f in X.columns]
        if features_to_drop:
            X = X.drop(columns=features_to_drop)
            print(f"已从数据中移除特征: {', '.join(features_to_drop)}")
    
    numeric_cols = X.select_dtypes(include=['number']).columns.tolist()
    return X[numeric_cols], y

def train_and_evaluate(X_train, y_train, X_test, y_test, params):
    """训练模型并返回评估结果"""
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)
    
    model = xgb.train(
        params,
        dtrain,
        num_boost_round=1000,
        evals=[(dtrain, 'train'), (dtest, 'test')],
        early_stopping_rounds=50,
        verbose_eval=False
    )
    
    # 评估模型
    ddata_train = xgb.DMatrix(X_train)
    ddata_test = xgb.DMatrix(X_test)
    
    y_pred_train = model.predict(ddata_train)
    y_pred_test = model.predict(ddata_test)
    
    metrics = {
        'Train R2': r2_score(y_train, y_pred_train),
        'Test R2': r2_score(y_test, y_pred_test)
    }
    return model, metrics

# ==================== 主程序：消融分析 ====================
def main():
    os.makedirs(output_dir, exist_ok=True)
    
    print("="*60)
    print("特征消融分析 (Feature Ablation Analysis)")
    print("="*60)

    # 1. 加载完整数据并划分训练集和测试集
    # 这一步非常关键：我们在完整数据集上划分一次，
    # 后续所有的消融实验都将使用这个固定的划分，以保证结果的可比性。
    print("\n步骤 1/3: 加载完整数据并划分训练/测试集...")
    X_full, y_full = prepare_data(input_path, target_column)
    X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(
        X_full, y_full, test_size=outer_test_size, random_state=random_state
    )
    print(f"训练集大小: {X_train_full.shape}, 测试集大小: {X_test_full.shape}")

    # 2. 训练基准模型 (使用所有特征)
    print("\n步骤 2/3: 训练基准模型 (使用所有特征)...")
    base_model, base_metrics = train_and_evaluate(
        X_train_full, y_train_full, X_test_full, y_test_full, model_params
    )
    joblib.dump(base_model, os.path.join(output_dir, "base_model_with_all_features.pkl"))
    
    print("\n基准模型性能 (使用所有特征):")
    print(f"  - 训练集 R²: {base_metrics['Train R2']:.4f}")
    print(f"  - 测试集 R²: {base_metrics['Test R2']:.4f}")

    # 3. 开始消融实验
    print("\n步骤 3/3: 开始依次移除特征并重新训练...")
    results = []
    
    # 记录基准结果
    results.append({
        'Removed_Feature': 'None (Base Model)',
        'Remaining_Features': X_full.shape[1],
        'Train_R2': base_metrics['Train R2'],
        'Test_R2': base_metrics['Test R2'],
        'R2_Change_From_Base': 0.0
    })

    for feature in features_to_ablate:
        print(f"\n--- 正在移除特征: '{feature}' ---")
        
        # 从训练集和测试集中同时移除该特征
        if feature in X_train_full.columns:
            X_train_ablated = X_train_full.drop(columns=[feature])
            X_test_ablated = X_test_full.drop(columns=[feature])
        else:
            print(f"警告: 特征 '{feature}' 不在数据集中，跳过此次实验。")
            continue

        # 训练消融后的模型
        ablated_model, ablated_metrics = train_and_evaluate(
            X_train_ablated, y_train_full, X_test_ablated, y_test_full, model_params
        )
        
        # 保存模型
        joblib.dump(ablated_model, os.path.join(output_dir, f"model_without_{feature.replace('.', '_')}.pkl"))
        
        # 计算与基准模型的 R² 变化
        r2_change = ablated_metrics['Test R2'] - base_metrics['Test R2']
        
        print(f"  - 剩余特征数: {X_train_ablated.shape[1]}")
        print(f"  - 训练集 R²: {ablated_metrics['Train R2']:.4f}")
        print(f"  - 测试集 R²: {ablated_metrics['Test R2']:.4f}")
        print(f"  - 与基准相比 R² 变化: {r2_change:.4f}")

        # 记录结果
        results.append({
            'Removed_Feature': feature,
            'Remaining_Features': X_train_ablated.shape[1],
            'Train_R2': ablated_metrics['Train R2'],
            'Test_R2': ablated_metrics['Test R2'],
            'R2_Change_From_Base': r2_change
        })

    # 4. 汇总并保存结果
    print("\n" + "="*60)
    print("消融分析结果汇总")
    print("="*60)
    results_df = pd.DataFrame(results)
    print(results_df.to_string(index=False))

    # 保存详细结果到CSV文件
    results_df.to_csv(os.path.join(output_dir, "ablation_analysis_results.csv"), index=False, encoding='utf-8-sig')
    print(f"\n详细结果已保存至: {os.path.join(output_dir, 'ablation_analysis_results.csv')}")

    # 打印关键发现
    print("\n--- 关键发现 ---")
    # 找出对测试集性能影响最大的特征（R² 下降最多）
    worst_removal = results_df.iloc[1:]['R2_Change_From_Base'].idxmin()
    best_removal = results_df.iloc[1:]['R2_Change_From_Base'].idxmax()
    
    print(f"1. 移除后性能下降最显著的特征: '{results_df.loc[worst_removal, 'Removed_Feature']}'")
    print(f"   - R² 变化: {results_df.loc[worst_removal, 'R2_Change_From_Base']:.4f}")
    print(f"   - 该特征对模型性能至关重要。")

    print(f"\n2. 移除后性能下降最少（或提升）的特征: '{results_df.loc[best_removal, 'Removed_Feature']}'")
    print(f"   - R² 变化: {results_df.loc[best_removal, 'R2_Change_From_Base']:.4f}")
    print(f"   - 该特征可能是冗余的，或其信息可被其他特征替代。")

if __name__ == "__main__":
    main()

特征消融分析 (Feature Ablation Analysis)

步骤 1/3: 加载完整数据并划分训练/测试集...
训练集大小: (193, 19), 测试集大小: (49, 19)

步骤 2/3: 训练基准模型 (使用所有特征)...

基准模型性能 (使用所有特征):
  - 训练集 R²: 0.9989
  - 测试集 R²: 0.7779

步骤 3/3: 开始依次移除特征并重新训练...

--- 正在移除特征: 'EN.' ---
  - 剩余特征数: 18
  - 训练集 R²: 0.9989
  - 测试集 R²: 0.7665
  - 与基准相比 R² 变化: -0.0114

--- 正在移除特征: 'I.R.' ---
  - 剩余特征数: 18
  - 训练集 R²: 0.9989
  - 测试集 R²: 0.7766
  - 与基准相比 R² 变化: -0.0013

--- 正在移除特征: 'I.E.' ---
  - 剩余特征数: 18
  - 训练集 R²: 0.9992
  - 测试集 R²: 0.7690
  - 与基准相比 R² 变化: -0.0089

--- 正在移除特征: 'Density' ---
  - 剩余特征数: 18
  - 训练集 R²: 0.9989
  - 测试集 R²: 0.7907
  - 与基准相比 R² 变化: 0.0128

--- 正在移除特征: 'M.P.' ---
  - 剩余特征数: 18
  - 训练集 R²: 0.9987
  - 测试集 R²: 0.7848
  - 与基准相比 R² 变化: 0.0069

--- 正在移除特征: 'T.C.' ---
  - 剩余特征数: 18
  - 训练集 R²: 0.9979
  - 测试集 R²: 0.7919
  - 与基准相比 R² 变化: 0.0140

消融分析结果汇总
  Removed_Feature  Remaining_Features  Train_R2  Test_R2  R2_Change_From_Base
None (Base Model)                  19  0.998865 0.777910             0.000000
              EN.     

In [4]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.stats import spearmanr, kendalltau, pearsonr
import matplotlib.pyplot as plt
import os
import joblib

# ==================== 核心参数配置 ====================
# 数据和输出路径
input_path = "归一化数据-20251120.xlsx"
output_dir = "EN_statistical_complete_analysis"
model_path = "xgboost_model_ncv/xgboost_model.pkl"

# 关键列名定义
target_column = "SO2 tolerance"
feature_column = "EN."
en_range = (0.74, 0.78)

# 图表样式配置
figsize = (14, 10)
title_font = {'family': 'Arial', 'size': 24, 'weight': 'bold'}
label_font = {'family': 'Arial', 'size': 24, 'weight': 'bold'}
tick_font = {'family': 'Arial', 'size': 22, 'weight': 'bold'}
legend_font = {'family': 'Arial', 'size': 22, 'weight': 'bold'}

# 颜色配置
scatter_color = '#2E86AB'
trend_line_color = '#A23B72'

# 线条样式
line_width = 3
scatter_size = 80
border_width = 2
tick_width = 2
tick_length = 6
dpi = 600

# ==================== 工具函数定义 ====================
def create_output_dir():
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    print(f"输出目录已创建/确认：{output_dir}")

def load_and_preprocess_data():
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"输入数据文件不存在：{input_path}")
    df = pd.read_excel(input_path)
    
    required_cols = [target_column, feature_column]
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        raise ValueError(f"数据文件缺失必要列：{', '.join(missing_cols)}")
    
    df_clean = df.dropna(subset=required_cols).copy()
    print(f"原始数据总量：{len(df)}，去除缺失值后：{len(df_clean)}")
    
    df_target = df_clean[
        (df_clean[feature_column] >= en_range[0]) & 
        (df_clean[feature_column] <= en_range[1])
    ].copy()
    
    if len(df_target) < 5:
        raise Warning(f"目标区间样本量较少（{len(df_target)}个），统计结果需谨慎解读")
    elif len(df_target) < 3:
        raise ValueError(f"目标区间样本量不足（仅{len(df_target)}个），无法进行有效统计检验")
    
    print(f"目标区间（{en_range[0]}-{en_range[1]}）样本量：{len(df_target)}")
    return df_clean, df_target

def statistical_tests(df_target):
    x = df_target[feature_column].values
    y = df_target[target_column].values
    
    spearman_r, spearman_p = spearmanr(x, y)
    kendall_tau, kendall_p = kendalltau(x, y)
    pearson_r, pearson_p = pearsonr(x, y)
    
    slope, intercept, r_value, p_value_reg, std_err = stats.linregress(x, y)
    
    conf_int = stats.t.interval(
        0.95, len(x)-2, loc=slope, scale=std_err
    )
    
    results = {
        "样本量": len(df_target),
        "EN.区间": f"{en_range[0]}-{en_range[1]}",
        "Spearman相关系数(r)": round(spearman_r, 4),
        "Spearman p值": round(spearman_p, 6),
        "Kendall τ系数": round(kendall_tau, 4),
        "Kendall p值": round(kendall_p, 6),
        "Pearson相关系数(r)": round(pearson_r, 4),
        "Pearson p值": round(pearson_p, 6),
        "线性回归斜率": round(slope, 4),
        "斜率p值": round(p_value_reg, 6),
        "斜率95%置信区间": (round(conf_int[0], 4), round(conf_int[1], 4)),
        "R²（决定系数）": round(r_value**2, 4),
        "统计显著性结论": "显著单调正相关" if (spearman_p < 0.05 and spearman_r > 0) else "不显著"
    }
    
    return results, x, y

def plot_comprehensive_analysis(x, y, results):
    create_output_dir()
    fig, ax = plt.subplots(figsize=figsize, facecolor='white')
    
    # 绘制散点图
    ax.scatter(x, y, color=scatter_color, s=scatter_size, alpha=0.8, 
               edgecolors='black', linewidth=1.5, label='Original Data')
    
    # 绘制趋势线
    z = np.polyfit(x, y, 1)
    p = np.poly1d(z)
    x_trend = np.linspace(en_range[0], en_range[1], 100)
    ax.plot(x_trend, p(x_trend), color=trend_line_color, linewidth=line_width, 
            alpha=0.9, label=f'Trend Line (r={results["Spearman相关系数(r)"]:.3f})')
    
    # 设置坐标轴范围
    ax.set_xlim(en_range[0]-0.01, en_range[1]+0.01)
    ax.set_ylim(y.min()-0.05, y.max()+0.05)
    
    # 设置标题
    title_text = f'{feature_column} vs NOₓ Conversion (0.74-0.78 Interval)\n' \
                 f'Spearman p={results["Spearman p值"]:.6f}, Kendall p={results["Kendall p值"]:.6f}'
    ax.set_title(title_text, fontdict=title_font, pad=25)
    ax.set_xlabel(f'{feature_column} ', fontdict=label_font, labelpad=15)
    ax.set_ylabel('NOₓ Conversion', fontdict=label_font, labelpad=15)
    
    # 设置刻度样式
    ax.tick_params(axis='both', which='major', width=tick_width, length=tick_length, 
                   labelsize=tick_font['size'])
    for label in ax.get_xticklabels():
        label.set_fontproperties(tick_font)
    for label in ax.get_yticklabels():
        label.set_fontproperties(tick_font)
    
    # 设置边框样式
    for spine in ax.spines.values():
        spine.set_linewidth(border_width)
        spine.set_color('black')
    
    # 设置图例
    ax.legend(prop=legend_font, frameon=False, loc='upper left', bbox_to_anchor=(0.02, 0.98))
    
    # 添加网格
    ax.grid(True, alpha=0.3, linestyle='-', linewidth=0.8)
    
    # 保存图片
    save_path = os.path.join(output_dir, f"{feature_column}_comprehensive_analysis.png")
    plt.tight_layout()
    plt.savefig(save_path, dpi=dpi, bbox_inches='tight', facecolor='white')
    plt.close()
    print(f"综合分析图已保存：{save_path}")

def save_statistical_report(results):
    create_output_dir()
    report_path = os.path.join(output_dir, f"{feature_column}_statistical_report.txt")
    
    with open(report_path, 'w', encoding='utf-8') as f:
        f.write("="*80 + "\n")
        f.write("EN.与NOₓ转化率单调正相关统计显著性分析报告\n")
        f.write("="*80 + "\n\n")
        
        f.write("1. 分析基础信息\n")
        f.write("-"*40 + "\n")
        f.write(f"数据文件：{input_path}\n")
        f.write(f"分析区间：EN. ∈ [{en_range[0]}, {en_range[1]}]\n")
        f.write(f"目标变量：{target_column}（NOₓ转化率）\n")
        f.write(f"分析样本量：{results['样本量']}个\n\n")
        
        f.write("2. 核心统计检验结果\n")
        f.write("-"*40 + "\n")
        f.write(f"Spearman秩相关系数 (r)：{results['Spearman相关系数(r)']}\n")
        f.write(f"Spearman p值：{results['Spearman p值']} {'(显著)' if results['Spearman p值'] < 0.05 else '(不显著)'}\n")
        f.write(f"Kendall τ系数：{results['Kendall τ系数']}\n")
        f.write(f"Kendall p值：{results['Kendall p值']} {'(显著)' if results['Kendall p值'] < 0.05 else '(不显著)'}\n")
        f.write(f"Pearson相关系数 (r)：{results['Pearson相关系数(r)']}\n")
        f.write(f"Pearson p值：{results['Pearson p值']} {'(显著)' if results['Pearson p值'] < 0.05 else '(不显著)'}\n\n")
        
        f.write("3. 线性回归分析结果\n")
        f.write("-"*40 + "\n")
        f.write(f"回归斜率：{results['线性回归斜率']}\n")
        f.write(f"斜率p值：{results['斜率p值']}\n")
        f.write(f"斜率95%置信区间：{results['斜率95%置信区间']}\n")
        f.write(f"决定系数 (R²)：{results['R²（决定系数）']}\n\n")
        
        f.write("4. 结论\n")
        f.write("-"*40 + "\n")
        if results['统计显著性结论'] == "显著单调正相关":
            f.write(f"在EN. ∈ [{en_range[0]}, {en_range[1]}]区间内，\n")
            f.write(f"EN.与NOₓ转化率呈显著单调正相关（Spearman r={results['Spearman相关系数(r)']:.4f}, p={results['Spearman p值']:.6f} < 0.05）。\n")
            f.write(f"线性回归斜率为正（{results['线性回归斜率']:.4f}），且斜率显著不为0（p={results['斜率p值']:.6f}），\n")
            f.write("进一步验证了正相关趋势的可靠性。")
        else:
            f.write(f"在EN. ∈ [{en_range[0]}, {en_range[1]}]区间内，\n")
            f.write(f"EN.与NOₓ转化率的单调正相关不显著（Spearman p={results['Spearman p值']:.6f} ≥ 0.05），\n")
            f.write("需扩大样本量或检查数据分布合理性。")
    
    print(f"详细统计报告已保存：{report_path}")

def save_data_table(df_target, results):
    create_output_dir()
    table_path = os.path.join(output_dir, f"{feature_column}_target_range_data.xlsx")
    
    df_output = df_target[[feature_column, target_column]].copy()
    df_output.columns = [f"{feature_column} (Normalized)", "NOₓ Conversion (Normalized)"]
    df_output = df_output.sort_values(by=f"{feature_column} (Normalized)").reset_index(drop=True)
    
    summary_row = pd.Series({
        f"{feature_column} (Normalized)": "统计摘要",
        "NOₓ Conversion (Normalized)": f"Spearman r={results['Spearman相关系数(r)']:.4f}, p={results['Spearman p值']:.6f}"
    })
    df_output = pd.concat([df_output, summary_row.to_frame().T], ignore_index=True)
    
    df_output.to_excel(table_path, index=False)
    print(f"目标区间数据表格已保存：{table_path}")

# ==================== 主程序执行 ====================
if __name__ == "__main__":
    print("="*80)
    print("开始执行EN.与NOₓ转化率单调正相关统计显著性完整分析")
    print("="*80)
    
    try:
        create_output_dir()
        
        df_clean, df_target = load_and_preprocess_data()
        
        stats_results, x_data, y_data = statistical_tests(df_target)
        
        plot_comprehensive_analysis(x_data, y_data, stats_results)
        
        save_statistical_report(stats_results)
        
        save_data_table(df_target, stats_results)
        
        print("\n" + "="*80)
        print("核心结果摘要：")
        print("="*80)
        print(f"分析区间：EN. ∈ [{en_range[0]}, {en_range[1]}]")
        print(f"样本量：{stats_results['样本量']}个")
        print(f"Spearman相关系数：{stats_results['Spearman相关系数(r)']}，p值：{stats_results['Spearman p值']}")
        print(f"Kendall τ系数：{stats_results['Kendall τ系数']}，p值：{stats_results['Kendall p值']}")
        print(f"统计显著性结论：{stats_results['统计显著性结论']}")
        print("="*80)
        print(f"分析完成！所有结果已保存至：{output_dir}")
        
    except Exception as e:
        print(f"\n分析过程出错：{str(e)}")
        print("请检查输入文件路径、列名正确性或样本量是否充足")

开始执行EN.与NOₓ转化率单调正相关统计显著性完整分析
输出目录已创建/确认：EN_statistical_complete_analysis
原始数据总量：242，去除缺失值后：242
目标区间（0.74-0.78）样本量：80
输出目录已创建/确认：EN_statistical_complete_analysis
综合分析图已保存：EN_statistical_complete_analysis\EN._comprehensive_analysis.png
输出目录已创建/确认：EN_statistical_complete_analysis
详细统计报告已保存：EN_statistical_complete_analysis\EN._statistical_report.txt
输出目录已创建/确认：EN_statistical_complete_analysis
目标区间数据表格已保存：EN_statistical_complete_analysis\EN._target_range_data.xlsx

核心结果摘要：
分析区间：EN. ∈ [0.74, 0.78]
样本量：80个
Spearman相关系数：0.3776，p值：0.000554
Kendall τ系数：0.275，p值：0.000518
统计显著性结论：显著单调正相关
分析完成！所有结果已保存至：EN_statistical_complete_analysis
